In [ ]:
# dashboard/app.py
import streamlit as st
import geopandas as gpd
from streamlit_folium import st_folium
import folium
import joblib

st.set_page_config(page_title="GeoExplain Dashboard", layout="wide")
st.title("🌊 GeoExplain: Flood Risk Dashboard")

@st.cache_resource
def load_resources():
    gdf = gpd.read_parquet("data/mumbai_grid_enhanced_with_ensemble_probs.parquet")
    model = joblib.load("models/ensemble_rf_xgb_mvp.pkl")
    return gdf, model

gdf, model = load_resources()

tab1, tab2 = st.tabs(["Flood Risk Map", "SHAP Explain"])
with tab1:
    st.subheader("Flood Probability Map")
    m = folium.Map(location=[19.07, 72.88], zoom_start=11, tiles="CartoDB positron")
    folium.Choropleth(
        geo_data=gdf,
        data=gdf,
        columns=["grid_id", "FloodProbability"],
        key_on="feature.properties.grid_id",
        fill_color="YlOrRd",
        legend_name="Flood Probability",
        nan_fill_color="white"
    ).add_to(m)
    st_folium(m, width=900, height=600)

with tab2:
    st.subheader("SHAP Summary")
    st.image("dashboard/static/shap_summary.png")
    st.caption("SHAP summary generated from model explainability.")
